## Part 1

In [27]:
import re

In [136]:
TEST_INFILE = "inputs/day_19_test_1.txt"
INFILE = "inputs/day_19_1.txt"

with open(TEST_INFILE) as infile:
#with open(INFILE) as infile:
    input = infile.read()
rules = input.split("\n\n")[0].split()
parts = input.split("\n\n")[1].split()

In [128]:
def process_rule(rule):
    rule_name  = rule.split("{")[0]
    rule_logic = rule.split("{")[1].rstrip("}")

    rules = rule_logic.split(",")
    parsed_rules = []
    for r in rules:
        rs = r.split(":")
        if len(rs) == 2:
            parsed_rules.append((rs[0], rs[1]))
        else:
            parsed_rules.append(("True", rs[0]))

    return rule_name, parsed_rules

In [129]:
rules = [process_rule(r) for r in rules]
rules = {rule_name: rule_logic for rule_name, rule_logic in rules}

In [130]:
# make each part a dict we can eval
parts = [re.sub("(x|m|a|s)", lambda m: f"'{m.group(0)}'", p.replace("=", ":")) for p in parts]

In [ ]:
def process_part(part, debug=False):
    if debug: print(f"Examining part {part}")
    rule = "in"
    while True:
        if debug: print(f"Rule {rule}")
        sub_rules = rules[rule]
        for sub_rule in sub_rules:
            if debug: print(f"\tLooking at sub rule {sub_rule[0]} of {rule}")
            truth = eval(sub_rule[0], eval(part))
            if truth:
                if debug: print(f"\t Satisfied so => {sub_rule[1]}")
                rule = sub_rule[1]
                if rule == "A":
                    return ("A", eval("x+m+a+s", eval(part)))
                elif rule == "R":
                    return ("R", None)
                else:
                    break

In [132]:
results = [process_part(p) for p in parts]

In [133]:
sum(r[1] for r in results if r[0] == "A")

19114

## Part 2

In [326]:
from functools import reduce

In [381]:
TEST_INFILE = "inputs/day_19_test_1.txt"
INFILE = "inputs/day_19_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    input = infile.read()
rules = input.split("\n\n")[0].split()
parts = input.split("\n\n")[1].split()

In [382]:
class Range:
    def __init__(self, lo, hi, lo_included=False, hi_included=False):
        self.lo = lo
        self.hi = hi
        self.lo_included = lo_included
        self.hi_included = hi_included


    def __len__(self):
        hi = self.hi if self.hi_included else (self.hi - 1)
        lo = self.lo if self.lo_included else (self.lo + 1)
        return hi - lo + 1


    def __repr__(self):
        out = ""
        if self.lo_included:
            out += "["
        else:
            out += "("
        out += f"{self.lo}, {self.hi}"
        if self.hi_included:
            out += "]"
        else:
            out += ")"
        return out


    def __eq__(self, other):
        return self.lo == other.lo and self.hi == other.hi \
            and self.lo_included == other.lo_included \
            and self.hi_included == other.hi_included


    def __and__(self, other):
        if self.lo > other.hi or other.lo > self.hi:
            return None

        lo = max(self.lo, other.lo)
        hi = min(self.hi, other.hi)

        if lo == self.lo and lo != other.lo:
            lo_included = self.lo_included
        elif lo == other.lo and lo != self.lo:
            lo_included = other.lo_included
        else:
            lo_included = self.lo_included and other.lo_included

        if hi == self.hi and hi != other.hi:
            hi_included = self.hi_included
        elif hi == other.hi and hi != self.hi:
            hi_included = other.hi_included
        else:
            hi_included = self.hi_included and other.hi_included


        return Range(lo, hi, lo_included, hi_included)
    

assert str(Range(0, 1000)) == "(0, 1000)"
assert str(Range(0, 1000, lo_included=True)) == "[0, 1000)"
assert str(Range(0, 1000, hi_included=True)) == "(0, 1000]"
assert str(Range(-float("inf"), 1000)) == "(-inf, 1000)"

assert Range(-float("inf"), 10) & Range(5, float("inf")) == Range(5, 10)
assert Range(-float("inf"), 10) & Range(15, float("inf")) is None
assert Range(-float("inf"), 10) & Range(5, float(7)) == Range(5, 7)

assert Range(-float("inf"), 10) & Range(5, float("inf"), lo_included=True) == Range(5, 10, lo_included=True)
assert Range(-float("inf"), 10) & Range(5, float(7), lo_included=True, hi_included=True) == Range(5, 7, lo_included=True, hi_included=True)
assert Range(10, 15) & Range(10, 15, lo_included=True, hi_included=True) == Range(10, 15, lo_included=False, hi_included=False)

assert len(Range(1, 10)) == 8
assert len(Range(1, 10, lo_included=True)) == 9
assert len(Range(1, 10, hi_included=True)) == 9
assert len(Range(1, 10, lo_included=True, hi_included=True)) == 10

In [383]:
class RangeSet:
    def __init__(self, ranges=None):
        if not ranges:
            ranges = {
                "x": Range(-float("inf"), float("inf")),
                "m": Range(-float("inf"), float("inf")),
                "a": Range(-float("inf"), float("inf")),
                "s": Range(-float("inf"), float("inf"))
            }
            self.ranges = ranges
        else:
            self.ranges = ranges


    def __repr__(self):
        return f"RangeSet(ranges={self.ranges})"
    

    def __and__(self, other):
        out = RangeSet()
        for var in self.ranges.keys():
            out.ranges[var] = self.ranges[var] & other.ranges[var]
        return out
    
    def __len__(self):
        return len(self.ranges["x"]) * len(self.ranges["m"]) * len(self.ranges["a"]) * len(self.ranges["s"])

In [384]:
def process_rule(rule, debug=False):
    rule_name  = rule.split("{")[0]
    rule_logic = rule.split("{")[1].rstrip("}")

    rules = rule_logic.split(",")
    
    output = []
    prev = RangeSet()
    for rule in rules:
        if debug: print(f"Looking at rule {rule}")

        subrule = rule.split(":")
        subrule_yes = RangeSet()
        subrule_no  = RangeSet()
        
        # full rule, not the last default case
        if len(subrule) == 2:
            var, op, pred = re.split("(>|<)", subrule[0])
            if op == "<":
                rule_range     = Range(-float("inf"), int(pred))
                not_rule_range = Range(int(pred), float("inf"), lo_included=True)
            elif op == ">":
                rule_range     = Range(int(pred), float("inf"))
                not_rule_range = Range(-float("inf"), int(pred), hi_included=True)
            else:
                raise ValueError(f"Unknown operator {op} in rule {subrule[0]}!!")

            subrule_yes.ranges[var] = subrule_yes.ranges[var] & rule_range
            subrule_no.ranges[var]  = subrule_no.ranges[var]  & not_rule_range

            yes = prev & subrule_yes
            no  = prev & subrule_no

            output.append((subrule[1], yes))
            prev = no
        else:
            output.append((subrule[0], no))

    return rule_name, output

In [385]:
edges = {}
for r in rules:
    rule_name, output_edges = process_rule(r)
    edges[rule_name] = output_edges

In [386]:
def path_to_end(from_node, edges, path_to_node):
    paths = []
    for neighbor_node, edge_rangeset in edges[from_node]:
        if neighbor_node == "R":
            continue
        elif neighbor_node == "A":
            paths.append(path_to_node + [(neighbor_node, edge_rangeset)])
        else:
            path = path_to_end(neighbor_node, edges, path_to_node + [(neighbor_node, edge_rangeset)])
            if len(path):
                paths.extend(path)
    return paths

In [387]:
all_paths = path_to_end("in", edges, [])

In [388]:
parts_range = Range(1, 4000, lo_included=True, hi_included=True)
parts_rangeset = RangeSet(
    {
        "x": parts_range,
        "m": parts_range,
        "a": parts_range,
        "s": parts_range
    }
)

total = 0
for p in all_paths:
    for n in p:
        print(f"{n[0]}", end="")
        if n[0] != "A":
            print(" => ", end="")
    print("")
    path_rangeset = reduce(lambda x, y: x & y, [r[1] for r in p])
    parts = path_rangeset & parts_rangeset
    total += len(parts)
    print(f"\t{path_rangeset}")
    print(f"\t{parts}")
    print(f"\t{len(parts)}")

cd => qjl => zdd => nzs => svh => A
	RangeSet(ranges={'x': (-inf, 2228), 'm': (2131, inf), 'a': (2405, inf), 's': (911, 1624)})
	RangeSet(ranges={'x': [1, 2228), 'm': (2131, 4000], 'a': (2405, 4000], 's': (911, 1624)})
	4726832353320
cd => qjl => zdd => nzs => svh => A
	RangeSet(ranges={'x': (-inf, 2228), 'm': (-inf, 2131], 'a': (2405, inf), 's': (1205, 1624)})
	RangeSet(ranges={'x': [1, 2228), 'm': [1, 2131], 'a': (2405, 4000], 's': (1205, 1624)})
	3164030315270
cd => qjl => zdd => nzs => svh => sdj => A
	RangeSet(ranges={'x': (-inf, 2228), 'm': (1268, 2131], 'a': (2405, inf), 's': (911, 1205]})
	RangeSet(ranges={'x': [1, 2228), 'm': (1268, 2131], 'a': (2405, 4000], 's': (911, 1205]})
	901237035930
cd => qjl => zdd => nzs => svh => sdj => A
	RangeSet(ranges={'x': (-inf, 2228), 'm': (-inf, 1268], 'a': (2405, 3341), 's': [1037, 1205]})
	RangeSet(ranges={'x': [1, 2228), 'm': [1, 1268], 'a': (2405, 3341), 's': [1037, 1205]})
	446208445540
cd => qjl => zdd => nzs => drm => mx => A
	RangeSe

In [389]:
total

122112157518711